In [1]:
!pip install -q py-feat
# !pip install -q facial-emotion-recognition
# !pip install -q fer
# !pip install -q deepface
# !pip install -q mtcnn

     |████████████████████████████████| 6.0MB 3.6MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
     |████████████████████████████████| 57.4MB 84kB/s 
     |████████████████████████████████| 4.9MB 49.9MB/s 


In [18]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from feat import Detector
import glob
import math
import dlib
import joblib
import cv2
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from collections import Counter

import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

face_model = "RetinaFace"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

def my_mode(sample):
  c = Counter(sample)
  return [k for k, v in c.items() if v == c.most_common(1)[0][1]]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading Face Detection model:  RetinaFace
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/mobilenet0.25_Final.pth
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/mobilenet_224_model_best_gdconv_external.pth.tar
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_scalar_aus.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/svm_568.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downl

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## ALL STEP IN ONE

In [14]:
ls 'drive/MyDrive/file2/vdo/Isaree/01_karnmay.mp4'

drive/MyDrive/file2/vdo/Isaree/01_karnmay.mp4


In [11]:
ls 'drive/MyDrive/file2/done/face2_01_KARNMAY_P1.csv'

drive/MyDrive/file2/done/face2_01_KARNMAY_P1.csv


In [17]:
ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/model_FERG_DB_256'

model_4_1.h5    model_4_2_1.h5  model_4_2_3.h5  model_4.h5
model_4_2_0.h5  model_4_2_2.h5  model_4_2_4.h5  model_FERG_DB_256.ipynb


In [22]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

# Create  model
model_4 = Model(inputs=base_model.input, outputs=output)

base_model_layers = model_4.layers[:23]
new_model_layers =  model_4.layers[23:]

base_model_blocks = {
    0: base_model_layers[1:3],
    1: base_model_layers[4:6],
    2: base_model_layers[7:11],
    3: base_model_layers[12:16],
    4: base_model_layers[17:21]
}

# Load the best model
model_4_path = "./drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/model_FERG_DB_256/{}.h5".format("model_4")
model_4.load_weights(model_4_path)

In [ ]:
pred_emotion= []

filepath = 'drive/MyDrive/file2/done/face2_01_KARNMAY_P1.csv'
vdopath ='drive/MyDrive/file2/vdo/Isaree/01_karnmay.mp4'
df = pd.read_csv(filepath)
dim = (224, 224)
rowimg = []
label_img = []
print('df', df.shape)

for i in tqdm(range(df.shape[0])):
  seqimg = []
  for j in range(df['start_frame'][i], df['end_frame'][i]):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    try:
      pos = faceimg[0][:4]
      cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] 
      resized = cv2.resize(cropimg, dim, interpolation=cv2.INTER_CUBIC)     
      image = preprocess_input(resized)
      image = np.expand_dims(image, axis=0)
      preds = model_4.predict(image)
      pred_class = np.argmax(preds)
      seqimg.append(pred_class) 
    except:
      print("Emotion Error  i = ",i, ' j =', j)

  seq_label = my_mode(seqimg)
  print('i =', i, 'seq_label =', seq_label)
  pred_emotion.append(seq_label[0])

df (152, 8)


In [ ]:
while True:pass

In [ ]:
dic_resmask =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_kagle =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_ckplus =  {1:'anger',3:'disgust', 4:'fear', 5:'happiness',6:'sadness',	7:'surprise'}
# dic_fer_svm_rf =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_fer_svm_rf =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_Deepfake =  {0:'anger',1:'fear', 2:'neutral', 3:'sad',4:'disgust',	5:'happy',	6:'surprise' }

pred = pd.DataFrame({'Pred':pred_emotion})
pred['emotion'] = pred['Pred'].replace(dic_resmask) 
result = pd.concat([df, pred], axis=1, join="inner")

from google.colab import files
result.to_csv('05_pimrypie_tony_R1.csv') 
files.download('05_pimrypie_tony_R1.csv')